In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('../assets/data/stage/stage_prontuario_prematuros_bkp.csv')

In [44]:
def substituir_dheg_pe(morbidade):
    if isinstance(morbidade, str):
        valor_original = morbidade
        morbidades = morbidade.split(', ')


        if "DHEG" in morbidades and "PE" in morbidades:
            morbidades = [m for m in morbidades if m not in ["DHEG", "PE"]]
            morbidades.append("PE")

        # Corrige a lógica: remover apenas se ambos existirem
        elif "HAS PREVIA" in morbidades and "PE" in morbidades:
            morbidades = [m for m in morbidades if m not in ["HAS PREVIA", "PE"]]
            morbidades.append("HAS CRONICA + DHEG SOBREPOSTA")

        elif "DHEG" in morbidades:
            morbidades = ["HIPERTENSAO SEM PROTEINURIA" if m == "DHEG" else m for m in morbidades]
        
        elif "HAS PREVIA" in morbidades:
            morbidades = ["HIPERTENSAO SEM PROTEINURIA" if m == "HAS PREVIA" else m for m in morbidades]


        valor_modificado = ', '.join(morbidades)
        return valor_modificado, valor_modificado != valor_original
    else:
        return morbidade, False

# Aplica a função à coluna 'MORBIDADE'
df[['MORBIDADE_MODIFICADA', 'ALTERADO']] = df['MORBIDADE'].apply(substituir_dheg_pe).apply(pd.Series)


In [45]:
cols = [col for col in df.columns if col not in ['MORBIDADE', 'MORBIDADE_MODIFICADA']] + ['MORBIDADE', 'MORBIDADE_MODIFICADA']
df = df[cols]

# Ordenar o DataFrame pela coluna 'ALTERADO' (True primeiro)
df_sorted = df.sort_values(by='ALTERADO', ascending=False)


In [46]:
df_sorted_drop_morbidade = df_sorted.drop(columns=['MORBIDADE', 'ALTERADO'])
df_sorted_drop_morbidade_renomeado = df_sorted_drop_morbidade.rename(columns={'MORBIDADE_MODIFICADA': 'MORBIDADE'})

In [47]:
df_sorted_drop_morbidade_renomeado.to_csv('../assets/data/stage/stage_prontuario_prematuros.csv', index=False)